# DT Assignment1

# Data Loading

In [1]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


In [3]:
pd_data.shape

(14, 5)

# 1. Gini 계수를 구하는 함수 만들기

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [22]:
def get_gini(df, label):
    gini = 1
    n = len(df)
    for i in df[label].value_counts():
        gini -= (i/n)**2
    return gini    
# 데이터프레임과 정답 컬럼만 주면 지니값구한다.

In [23]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
 ## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

In [36]:
def get_binary_split(df, attribute):
    result = []
    classes = list(df[attribute].unique())
    for i in classes:
        tmp_class = classes.copy()
        result.append([i])
        tmp_class.remove(i)
        result.append(tmp_class)
    return result


In [37]:
get_binary_split(pd_data,"age")

[['youth'],
 ['middle_aged', 'senior'],
 ['middle_aged'],
 ['youth', 'senior'],
 ['senior'],
 ['youth', 'middle_aged']]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [78]:
def get_attribute_gini_index(df, attribute, label):
    result = {}
    n = len(df)
    classes = get_binary_split(df, attribute)
    for i in classes:
        if len(i)==1:
            tmp_df0 = df[df[attribute]==i[0]]
            tmp_df1 = df[df[attribute]!=i[0]]
            value = len(tmp_df0)/n*get_gini(tmp_df0,label) + len(tmp_df1)/n*get_gini(tmp_df1,label)
            result[i[0]] = value
        
    for i in classes:
        if len(i) == 1:
            for ii in classes:
                if len(ii) != 1 and i[0] not in ii:
                    key = '_'.join(ii)
                    result[key] = result[i[0]]
    return result

In [79]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'youth': 0.3936507936507937,
 'middle_aged': 0.35714285714285715,
 'senior': 0.4571428571428572,
 'middle_aged_senior': 0.3936507936507937,
 'youth_senior': 0.35714285714285715,
 'youth_middle_aged': 0.4571428571428572}

여기서 가장 작은 Gini index값을 가지는 class를 기준으로 split해야겠죠?

In [48]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())

('middle_aged', 0.35714285714285715)

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) 변수 ‘income’의 이진분류 결과를 보여주세요.

## 문제2) 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.

## 문제3) 문제 2에서 제시한 feature로 DataFrame을 split한 후 나눠진 2개의 DataFrame에서 각각   다음으로 중요한 변수를 선정하고 해당 변수의 Gini index를 제시해주세요.

In [118]:
##문제1 답안
def split(df, attribute, cla): # 데이터프레임을 두개로 분리하는 함수
    a = pd_data[pd_data[attribute]==cla]
    b = pd_data[pd_data[attribute]!=cla]
    return a, b 
m = min(get_attribute_gini_index(pd_data, "income", "class_buys_computer").items())[0]
a, b = split(pd_data, 'income', m)
print(a)
print('-'*100)
print(b)

            age income student credit_rating class_buys_computer
0         youth   high      no          fair                  no
1         youth   high      no     excellent                  no
2   middle_aged   high      no          fair                 yes
12  middle_aged   high     yes          fair                 yes
----------------------------------------------------------------------------------------------------
            age  income student credit_rating class_buys_computer
3        senior  medium      no          fair                 yes
4        senior     low     yes          fair                 yes
5        senior     low     yes     excellent                  no
6   middle_aged     low     yes     excellent                 yes
7         youth  medium      no          fair                  no
8         youth     low     yes          fair                 yes
9        senior  medium     yes          fair                 yes
10        youth  medium     yes     excellent 

In [119]:
##문제2 답안
lst = []
for i in pd_data.columns[:-1]:
    m = min(get_attribute_gini_index(pd_data, i, "class_buys_computer").items())
    lst.append(m)
print(lst)
# mm = min([i[1] for i in lst])
mm = min(lst, key=lambda x : x[1])
print(mm)

[('middle_aged', 0.35714285714285715), ('high', 0.4428571428571429), ('no', 0.3673469387755103), ('excellent', 0.42857142857142855)]
('middle_aged', 0.35714285714285715)


1차 분리에서 제일 중요한 변수는 'age'의 'middle_aged' 클래스이다. 해당 변수의 Gini index는 0.35714285714285715이다.

In [120]:
##문제3 답안
a, b = split(pd_data, 'age', 'middle_aged')
print(a)
print('-'*100)
print(b)

            age  income student credit_rating class_buys_computer
2   middle_aged    high      no          fair                 yes
6   middle_aged     low     yes     excellent                 yes
11  middle_aged  medium      no     excellent                 yes
12  middle_aged    high     yes          fair                 yes
----------------------------------------------------------------------------------------------------
       age  income student credit_rating class_buys_computer
0    youth    high      no          fair                  no
1    youth    high      no     excellent                  no
3   senior  medium      no          fair                 yes
4   senior     low     yes          fair                 yes
5   senior     low     yes     excellent                  no
7    youth  medium      no          fair                  no
8    youth     low     yes          fair                 yes
9   senior  medium     yes          fair                 yes
10   youth  medium  

middle_aged 클래스는 모두 yes로 더 분리가 필요하지 않다.

In [123]:
lst = []
for i in pd_data.columns[:-1]:
    m = min(get_attribute_gini_index(b, i, "class_buys_computer").items())
    lst.append(m)
print(lst)
mm = min(lst, key=lambda x : x[1])
print(mm)

[('senior', 0.48), ('high', 0.375), ('no', 0.31999999999999984), ('excellent', 0.4166666666666667)]
('no', 0.31999999999999984)


가장 작은 Gini index를 가지는 'student'변수의 'no' 클래스로 분리한다.

In [124]:
a, b = split(b, 'student', 'no')
print(a)
print('-'*100)
print(b)

            age  income student credit_rating class_buys_computer
0         youth    high      no          fair                  no
1         youth    high      no     excellent                  no
2   middle_aged    high      no          fair                 yes
3        senior  medium      no          fair                 yes
7         youth  medium      no          fair                  no
11  middle_aged  medium      no     excellent                 yes
13       senior  medium      no     excellent                  no
----------------------------------------------------------------------------------------------------
            age  income student credit_rating class_buys_computer
4        senior     low     yes          fair                 yes
5        senior     low     yes     excellent                  no
6   middle_aged     low     yes     excellent                 yes
8         youth     low     yes          fair                 yes
9        senior  medium     yes          

2차 분리 결과는 위와 같다. 더욱 분리가 잘된 모습을 볼 수 있다.